# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Chokurdakh,RU,70.6333,147.9167,-15.83,100,58,5.06,broken clouds
1,1,Bac Lieu,VN,9.2850,105.7244,79.03,81,100,7.43,overcast clouds
2,2,Kamaishi,JP,39.2667,141.8833,57.85,64,91,8.70,overcast clouds
3,3,Rikitea,PF,-23.1203,-134.9692,71.44,79,30,13.76,light rain
4,4,Pisco,PE,-13.7000,-76.2167,61.02,88,0,9.22,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 18
What is the maximum temperature you would like for your vacation? 45


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [7]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)] 
                                    
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
13,13,Severo-Kurilsk,RU,50.6789,156.1250,38.44,71,6,13.49,clear sky
15,15,Khatanga,RU,71.9667,102.5000,24.57,93,100,21.23,light snow
35,35,Necochea,AR,-38.5473,-58.7368,44.74,70,5,7.47,clear sky
44,44,Berlevag,NO,70.8578,29.0864,37.40,75,100,9.22,overcast clouds
46,46,Upernavik,GL,72.7868,-56.1549,18.32,66,90,3.36,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [8]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
13,13,Severo-Kurilsk,RU,50.6789,156.1250,38.44,71,6,13.49,clear sky
15,15,Khatanga,RU,71.9667,102.5000,24.57,93,100,21.23,light snow
35,35,Necochea,AR,-38.5473,-58.7368,44.74,70,5,7.47,clear sky
44,44,Berlevag,NO,70.8578,29.0864,37.40,75,100,9.22,overcast clouds
46,46,Upernavik,GL,72.7868,-56.1549,18.32,66,90,3.36,overcast clouds


In [11]:
# Count how many City-Hotels are in the DataFrame
len(clean_travel_cities_df)

129

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [12]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head

<bound method NDFrame.head of                City Country  Max Temp       Description      Lat       Lng
13   Severo-Kurilsk      RU     38.44         clear sky  50.6789  156.1250
15         Khatanga      RU     24.57        light snow  71.9667  102.5000
35         Necochea      AR     44.74         clear sky -38.5473  -58.7368
44         Berlevag      NO     37.40   overcast clouds  70.8578   29.0864
46        Upernavik      GL     18.32   overcast clouds  72.7868  -56.1549
..              ...     ...       ...               ...      ...       ...
681          Jidong      CN     42.13  scattered clouds  45.2167  131.0833
686            Atig      RU     36.57        light rain  56.6931   59.4227
688         Atbasar      KZ     39.24   overcast clouds  51.8000   68.3333
693      Vanderhoof      CA     30.97     broken clouds  54.0166 -124.0198
697      Toora-Khem      RU     21.40   overcast clouds  52.4675   96.1100

[129 rows x 6 columns]>

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [13]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
13,Severo-Kurilsk,RU,38.44,clear sky,50.6789,156.1250,
15,Khatanga,RU,24.57,light snow,71.9667,102.5000,
35,Necochea,AR,44.74,clear sky,-38.5473,-58.7368,
44,Berlevag,NO,37.40,overcast clouds,70.8578,29.0864,
46,Upernavik,GL,18.32,overcast clouds,72.7868,-56.1549,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [14]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [15]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"] 
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
13,Severo-Kurilsk,RU,38.44,clear sky,50.6789,156.1250,Kak Doma
15,Khatanga,RU,24.57,light snow,71.9667,102.5000,"""Khatanga"""
35,Necochea,AR,44.74,clear sky,-38.5473,-58.7368,dyd -mar Hotel
44,Berlevag,NO,37.40,overcast clouds,70.8578,29.0864,Kvitbrakka
46,Upernavik,GL,18.32,overcast clouds,72.7868,-56.1549,Café de Upernavik


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [17]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
13,Severo-Kurilsk,RU,38.44,clear sky,50.6789,156.1250,Kak Doma
15,Khatanga,RU,24.57,light snow,71.9667,102.5000,"""Khatanga"""
35,Necochea,AR,44.74,clear sky,-38.5473,-58.7368,dyd -mar Hotel
44,Berlevag,NO,37.40,overcast clouds,70.8578,29.0864,Kvitbrakka
46,Upernavik,GL,18.32,overcast clouds,72.7868,-56.1549,Café de Upernavik


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [18]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [19]:
# Set the file name.
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [24]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [26]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Kak Doma</dd>\n<dt>City</dt><dd>Severo-Kurilsk</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>clear sky and 38.44 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>"Khatanga"</dd>\n<dt>City</dt><dd>Khatanga</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Weather</dt><dd>light snow and 24.57 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>dyd -mar Hotel</dd>\n<dt>City</dt><dd>Necochea</dd>\n<dt>Country</dt><dd>AR</dd>\n<dt>Current Weather</dt><dd>clear sky and 44.74 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Kvitbrakka</dd>\n<dt>City</dt><dd>Berlevag</dd>\n<dt>Country</dt><dd>NO</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 37.4 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Café de Upernavik</dd>\n<dt>City</dt><dd>Upernavik</dd>\n<dt>Country</dt><dd>GL</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 18.32 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Skytterhuset</dd>\n<dt>City</dt><dd>Hammerfest</d

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [27]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
13,50.6789,156.1250
15,71.9667,102.5000
35,-38.5473,-58.7368
44,70.8578,29.0864
46,72.7868,-56.1549
49,70.6634,23.6821
50,64.5011,-165.4064
53,46.4856,135.8833
57,65.6145,-37.6368
58,-54.8000,-68.3000


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [28]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))